# import Dependencies

In [1]:
# Import dependencies for multiple regression
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import re


# Import dependencies for google maps API
#from config import g_key
import gmaps
import requests
import geopy.distance

# Import dependencies for classifier models
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Data Transformation

In [2]:
# Import csv files to DataFrames
catastro_df = pd.read_csv('../../db_catastro.csv')
starbucks_df = pd.read_csv('../../MexicoCityStarbucks.csv')
airbnb_df = pd.read_csv('../../Airbnb.csv')

In [21]:
airbnb_df

,last_scraped,name,neighbourhood_cleansed,latitude,longitude,price
0,9/22/2022,Villa Dante,Cuajimalpa de Morelos,19.382830,-99.271780,4001.0
1,9/22/2022,"Area Sta.Fe,IberoABC Hospital Rooms",Cuajimalpa de Morelos,19.353790,-99.284590,500.0
2,9/23/2022,Classic but Modern Apartment in Condesa,Cuauhtémoc,19.418240,-99.175600,599.0
3,9/23/2022,CONDESA HAUS B&B,Cuauhtémoc,19.411620,-99.177940,18000.0
4,9/22/2022,Live Mexico City Like a Small Town,Benito Juárez,19.384680,-99.177080,740.0
...,...,...,...,...,...,...
22943,9/22/2022,Depa seguro a 10 mns en auto del AICM,Gustavo A. Madero,19.465703,-99.051118,450.0
22944,9/22/2022,Comoda habitación con vista e iluminación natural,Miguel Hidalgo,19.421690,-99.245990,750.0
22945,9/22/2022,Beautiful house gardens & terraces,Miguel Hidalgo,19.430260,-99.238160,5041.0
22946,9/22/2022,Amplio Dpto a 5 min de Av P. de las Palmas/Par...,Miguel Hidalgo,19.427120,-99.224520,1100.0


In [22]:
starbucks_df

,Brand,Street Address,City,State/Province,Country,Postcode,Longitude,Latitude
0,Starbucks,"de la Industria Militar 1296,, Lomas de Chamiz...",Ciudad de Mexico,DIF,MX,5129,-99.26,19.39
1,Starbucks,"Homero n.503, Col. Chapultepec Morales, Del. M...",DF,DIF,MX,11570,-99.19,19.44
2,Starbucks,"Palanque s/n, Col. Narvarte",DF,DIF,MX,1030,-99.15,19.40
3,Starbucks,AV SAN JERONIMO 819 COL SAN JERONIMO,DISTRITO FEDERAL,DIF,MX,10200,-99.23,19.32
4,Starbucks,RENATO LEDUC 224 COL TORIELLO GUERRA,DISTRITO FEDERAL,DIF,MX,14050,-99.16,19.30
...,...,...,...,...,...,...,...,...
224,Starbucks,"Eje 1 Nte Mosqueta 259 Buenavista, Eje 1 Nte M...",Mexico DF,DIF,MX,6350,-99.15,19.45
225,Starbucks,"Shiller esquina Masaryk 111, Polanco, Shiller ...",Mexico DF,DIF,MX,11550,-99.19,19.43
226,Starbucks,"Calz. Las Aguilas 1953 Col. Axomiatla, Del. Al...",Mexico DF,DIF,MX,11520,-99.25,19.34
227,Starbucks,"Av. Leyes de Reforma esq. Baratillo, Col. Ejid...",Mexico DF,DIF,MX,9040,-99.09,19.38


In [23]:
catastro_df

,Unnamed: 0,nombre_col,num_code,prom_valor_unitario_suelo,latitud,longitud,alcaldia,estado
0,0,19 DE MAYO,1210,1404.400000,19.361649,-99.253530,Álvaro Obregón,CDMX
1,1,1RA VICTORIA,1150,2061.619503,19.386686,-99.201985,Álvaro Obregón,CDMX
2,2,1RA VICTORIA SECCION BOSQUES,1150,1788.360000,19.387826,-99.197446,Álvaro Obregón,CDMX
3,3,2DA JALALPA TEPITO (AMPL),1260,1063.626266,19.375080,-99.233736,Álvaro Obregón,CDMX
4,4,2DA EL PIRUL (AMPL),1210,1320.710000,19.379745,-99.242231,Álvaro Obregón,CDMX
...,...,...,...,...,...,...,...,...
1784,73,GUADALUPE,16900,420.703636,19.218264,-99.120799,Xochimilco,CDMX
1785,74,NATIVITAS LA JOYA (AMPL),16900,711.580672,19.237250,-99.093782,Xochimilco,CDMX
1786,75,SAN FRANCISCO TLALNEPANTLA (PBLO),16910,338.647387,19.197937,-99.122385,Xochimilco,CDMX
1787,76,SANTA CECILIA TEPETLAPA (PBLO),16880,534.238763,19.217187,-99.099325,Xochimilco,CDMX


# Parse data to locate nearby Airbnbs

In [24]:
# Create columns to store number of Airbnb and Starbucks nearby
catastro_df['no_airbnb'] = ''
catastro_df['no_starbucks'] = ''
catastro_df['airbnb_mean_price'] = ''
catastro_df = catastro_df.reset_index()

# Declare for loop to iterate over every row of catastro_df to compare distance radius
for df_index, row in catastro_df.iterrows():
    
    #Declare variable for name of the Neighbourhood
    nombre_col = row['nombre_col']
    # Declare counter variable to hold number of Airbnbs and Starbucks nearby
    no_airbnb = 0
    no_starbucks = 0
    airbnb_price = 0
    holder_price = 0
    mean_price = 0
    
    # Get Latitude and Longitude of the row
    lat_cat = row['latitud']
    lng_cat = row['longitud']
    
    # Pair coordinates
    catastro_loc = (lat_cat,lng_cat)

    # With every catastro_df row, iterate and compare distances between each row of Airbnbs
    for bnb_index, row in airbnb_df.iterrows():
        
        # Get Latitude and Longitude of the row
        lat_bnb = row['latitude']
        lng_bnb = row['longitude']
        
        # Pair coordinates
        airbnb_loc = (lat_bnb,lng_bnb)
        
        # Using geopy.distance module, compare both coordinate to determine distance between them
        distancia = geopy.distance.geodesic(catastro_loc, airbnb_loc).m
        
        # Create conditional to add 1 to the counter if Airbnb is within 500m of Catastro_df row location
        if distancia < 800:
            airbnb_price = row['price']
            holder_price = airbnb_price + holder_price
            no_airbnb = no_airbnb + 1
    
    # After parsing on every row, append number of airbnbs nearby
    catastro_df.loc[df_index, 'no_airbnb'] = no_airbnb
    if no_airbnb > 0:
        mean_price = holder_price / no_airbnb
        catastro_df.loc[df_index, 'airbnb_mean_price'] = mean_price
    
    # With every catastro_df row, iterate and compare distances between each row of Starbucks
    for starbucks_index, row in starbucks_df.iterrows():
        try: 
            # Get Latitude and Longitude of the row
            lat_starbucks = row['Latitude']
            lng_starbucks = row['Longitude']
        
            # Pair Coordinates
            starbucks_loc = (lat_starbucks,lng_starbucks)
        
            # Using geopy.distance module, compare both coordinate to determine distance between them
            distancia_starbucks = geopy.distance.geodesic(catastro_loc, starbucks_loc).m
        
            # Create conditional to add 1 to the counter if Starbucks is within 500m of Catastro_df row location
            if distancia_starbucks < 700:
                no_starbucks = no_starbucks + 1
                
        except(ValueError):
            print('Value Error for', row['Street Address'])
    
    # After parsing on every row, append number of Starbucks nearby
    catastro_df.loc[df_index, 'no_starbucks'] = no_starbucks
    print('-----------------------------------')
    print(nombre_col)
    print(f'#{no_airbnb} Airbnbs found')
    print(f'Airbnb average cost: ${mean_price}')
    print(f'#{no_starbucks} Starbucks found')
    print('-----------------------------------')

-----------------------------------
19 DE MAYO
#19 Airbnbs found
Airbnb average cost: $756.6842105263158
#0 Starbucks found
-----------------------------------
-----------------------------------
1RA VICTORIA
#29 Airbnbs found
Airbnb average cost: $1033.3103448275863
#0 Starbucks found
-----------------------------------
-----------------------------------
1RA VICTORIA SECCION BOSQUES
#74 Airbnbs found
Airbnb average cost: $944.6486486486486
#0 Starbucks found
-----------------------------------
-----------------------------------
2DA  JALALPA TEPITO (AMPL)
#3 Airbnbs found
Airbnb average cost: $506.3333333333333
#0 Starbucks found
-----------------------------------
-----------------------------------
2DA EL PIRUL (AMPL)
#11 Airbnbs found
Airbnb average cost: $878.5454545454545
#0 Starbucks found
-----------------------------------
-----------------------------------
ABRAHAM GONZALEZ
#12 Airbnbs found
Airbnb average cost: $672.25
#0 Starbucks found
-----------------------------------


-----------------------------------
CRISTO REY
#32 Airbnbs found
Airbnb average cost: $894.90625
#0 Starbucks found
-----------------------------------
-----------------------------------
DE TARANGO (RCNDA)
#13 Airbnbs found
Airbnb average cost: $1178.7692307692307
#0 Starbucks found
-----------------------------------
-----------------------------------
DESARROLLO URBANO
#13 Airbnbs found
Airbnb average cost: $1495.7692307692307
#0 Starbucks found
-----------------------------------
-----------------------------------
DOS RIOS
#18 Airbnbs found
Airbnb average cost: $761.7777777777778
#0 Starbucks found
-----------------------------------
-----------------------------------
EL ARBOL
#6 Airbnbs found
Airbnb average cost: $524.1666666666666
#0 Starbucks found
-----------------------------------
-----------------------------------
EL BATAN
#19 Airbnbs found
Airbnb average cost: $756.1052631578947
#0 Starbucks found
-----------------------------------
-----------------------------------
EL

-----------------------------------
LA CONCHITA
#10 Airbnbs found
Airbnb average cost: $718.0
#0 Starbucks found
-----------------------------------
-----------------------------------
LA HERRADURA
#24 Airbnbs found
Airbnb average cost: $697.5833333333334
#0 Starbucks found
-----------------------------------
-----------------------------------
LA HUERTA
#11 Airbnbs found
Airbnb average cost: $570.7272727272727
#0 Starbucks found
-----------------------------------
-----------------------------------
LA JOYA
#4 Airbnbs found
Airbnb average cost: $409.0
#0 Starbucks found
-----------------------------------
-----------------------------------
LA LOMA
#17 Airbnbs found
Airbnb average cost: $1698.2941176470588
#0 Starbucks found
-----------------------------------
-----------------------------------
LA MARTINICA
#9 Airbnbs found
Airbnb average cost: $1425.4444444444443
#0 Starbucks found
-----------------------------------
-----------------------------------
LA MEXICANA
#5 Airbnbs found
A

-----------------------------------
MARIA G DE GARCIA RUIZ
#7 Airbnbs found
Airbnb average cost: $556.5714285714286
#0 Starbucks found
-----------------------------------
-----------------------------------
MARTIRES DE TACUBAYA
#6 Airbnbs found
Airbnb average cost: $399.3333333333333
#0 Starbucks found
-----------------------------------
-----------------------------------
MERCED GOMEZ
#29 Airbnbs found
Airbnb average cost: $653.9310344827586
#1 Starbucks found
-----------------------------------
-----------------------------------
METROPOLITANO (U HAB)
#10 Airbnbs found
Airbnb average cost: $1463.4
#0 Starbucks found
-----------------------------------
-----------------------------------
MIGUEL GAONA ARMENTA
#40 Airbnbs found
Airbnb average cost: $717.625
#0 Starbucks found
-----------------------------------
-----------------------------------
MIGUEL HIDALGO
#32 Airbnbs found
Airbnb average cost: $16234.21875
#0 Starbucks found
-----------------------------------
--------------------

-----------------------------------
SAN GABRIEL
#102 Airbnbs found
Airbnb average cost: $1485.6666666666667
#1 Starbucks found
-----------------------------------
-----------------------------------
SAN PEDRO DE LOS PINOS
#156 Airbnbs found
Airbnb average cost: $890.7435897435897
#3 Starbucks found
-----------------------------------
-----------------------------------
SANTA FE
#18 Airbnbs found
Airbnb average cost: $935.8888888888889
#0 Starbucks found
-----------------------------------
-----------------------------------
SANTA FE (U HAB)
#9 Airbnbs found
Airbnb average cost: $754.4444444444445
#0 Starbucks found
-----------------------------------
-----------------------------------
SANTA FE KM 8.5
#57 Airbnbs found
Airbnb average cost: $1456.438596491228
#0 Starbucks found
-----------------------------------
-----------------------------------
SANTA LUCIA
#20 Airbnbs found
Airbnb average cost: $843.3
#0 Starbucks found
-----------------------------------
---------------------------

...

-----------------------------------
XAXALCO
#1 Airbnbs found
Airbnb average cost: $600.0
#0 Starbucks found
-----------------------------------
-----------------------------------
XAXALIPAC
#0 Airbnbs found
Airbnb average cost: $0
#0 Starbucks found
-----------------------------------
-----------------------------------
MIRADOR 1A SECC
#22 Airbnbs found
Airbnb average cost: $426.45454545454544
#0 Starbucks found
-----------------------------------
-----------------------------------
MA ESTHER ZUNO DE ECHEVERRIA-TLALPUENTE
#5 Airbnbs found
Airbnb average cost: $1170.0
#0 Starbucks found
-----------------------------------
-----------------------------------
SAN ANDRES TOTOLTEPEC (PBLO)
#4 Airbnbs found
Airbnb average cost: $330.25
#0 Starbucks found
-----------------------------------
-----------------------------------
TETENCO (PJE)
#10 Airbnbs found
Airbnb average cost: $467.2
#0 Starbucks found
-----------------------------------
-----------------------------------
TLALMILLE
#2 Airbn

-----------------------------------
VALLE VERDE
#0 Airbnbs found
Airbnb average cost: $0
#0 Starbucks found
-----------------------------------
-----------------------------------
SAN PEDRO MARTIR (PBLO)
#23 Airbnbs found
Airbnb average cost: $4448.608695652174
#0 Starbucks found
-----------------------------------
-----------------------------------
EJIDOS DE SAN PEDRO MARTIR II (SUR)
#15 Airbnbs found
Airbnb average cost: $525.6666666666666
#0 Starbucks found
-----------------------------------
-----------------------------------
MIRADOR DEL VALLE
#2 Airbnbs found
Airbnb average cost: $914.5
#0 Starbucks found
-----------------------------------
-----------------------------------
ARENAL GUADALUPE TLALPAN
#28 Airbnbs found
Airbnb average cost: $616.6428571428571
#0 Starbucks found
-----------------------------------
-----------------------------------
SAN MIGUEL TOPILEJO (PBLO)
#2 Airbnbs found
Airbnb average cost: $700.0
#0 Starbucks found
-----------------------------------
-------

-----------------------------------
MORELOS I
#11 Airbnbs found
Airbnb average cost: $564.9090909090909
#0 Starbucks found
-----------------------------------
-----------------------------------
AVIACION CIVIL
#42 Airbnbs found
Airbnb average cost: $596.8809523809524
#7 Starbucks found
-----------------------------------
-----------------------------------
CUATRO ARBOLES
#65 Airbnbs found
Airbnb average cost: $685.6769230769231
#7 Starbucks found
-----------------------------------
-----------------------------------
MOCTEZUMA 2A SECCION II
#67 Airbnbs found
Airbnb average cost: $759.1343283582089
#0 Starbucks found
-----------------------------------
-----------------------------------
AVIACION CIVIL (AMPL)
#42 Airbnbs found
Airbnb average cost: $596.2619047619048
#0 Starbucks found
-----------------------------------
-----------------------------------
5TO TRAMO DE 20 DE NOVIEMBRE
#20 Airbnbs found
Airbnb average cost: $561.95
#0 Starbucks found
-----------------------------------
--

-----------------------------------
LA GUADALUPITA (BARR)
#7 Airbnbs found
Airbnb average cost: $787.2857142857143
#0 Starbucks found
-----------------------------------
-----------------------------------
SAN DIEGO (BARR)
#7 Airbnbs found
Airbnb average cost: $787.2857142857143
#0 Starbucks found
-----------------------------------
-----------------------------------
SANTA CRUCITA (BARR)
#6 Airbnbs found
Airbnb average cost: $860.1666666666666
#0 Starbucks found
-----------------------------------
-----------------------------------
BELEM (BARR)
#7 Airbnbs found
Airbnb average cost: $1008.0
#0 Starbucks found
-----------------------------------
-----------------------------------
EL ROSARIO (BARR)
#6 Airbnbs found
Airbnb average cost: $860.1666666666666
#0 Starbucks found
-----------------------------------
-----------------------------------
JARDINES DEL SUR
#9 Airbnbs found
Airbnb average cost: $1984.0
#0 Starbucks found
-----------------------------------
--------------------------

-----------------------------------
CERRO GRANDE
#0 Airbnbs found
Airbnb average cost: $0
#0 Starbucks found
-----------------------------------
-----------------------------------
LA CAADA
#0 Airbnbs found
Airbnb average cost: $0
#0 Starbucks found
-----------------------------------
-----------------------------------
SANTIAGO TEPALCATLALPAN (PBLO)
#2 Airbnbs found
Airbnb average cost: $1024.5
#0 Starbucks found
-----------------------------------
-----------------------------------
SAN JOSE ZACATEPEC
#1 Airbnbs found
Airbnb average cost: $3000.0
#0 Starbucks found
-----------------------------------
-----------------------------------
SANTA CRUZ CHAVARRIETA
#1 Airbnbs found
Airbnb average cost: $3000.0
#0 Starbucks found
-----------------------------------
-----------------------------------
SANTA INES
#0 Airbnbs found
Airbnb average cost: $0
#0 Starbucks found
-----------------------------------
-----------------------------------
SAN MATEO XALPA (PBLO)
#1 Airbnbs found
Airbnb aver

In [25]:
catastro_df.to_csv('catastro_ml_model_radio700.csv', index = False)

In [26]:
catastro_df_ml['flag_airbnb'] = ''
catastro_df_ml['flag_starbucks'] = ''

for index, row in catastro_df_ml.iterrows():
    is_there_airbnb = row ['no_airbnb']
    is_there_starbucks = row['no_starbucks']
    
    if is_there_airbnb > 0:
        catastro_df.loc[index, 'flag_airbnb'] = 1
    else:
        catastro_df.loc[index, 'flag_airbnb'] = 0
        
    if is_there_starbucks > 0:
        catastro_df.loc[index, 'flag_starbucks'] = 1
    else:
        catastro_df.loc[index, 'flag_starbucks'] = 0

In [27]:
# Drop unneccesary "Unnamed" column
catastro_df_ml = catastro_df_ml.drop('Unnamed: 0', axis = 1)
catastro_df_ml = catastro_df_ml.drop('index', axis = 1)

# Write Catastro Final DataFrame to csv to store the dat
#catastro_df_ml.to_csv('catastro_ml_model.csv', index = False)

In [28]:
catastro_df_ml.head()

,nombre_col,num_code,prom_valor_unitario_suelo,latitud,longitud,alcaldia,estado,no_airbnb,no_starbucks,airbnb_mean_price,flag_airbnb,flag_starbucks
0,19 DE MAYO,1210,1404.400000,19.361649,-99.253530,Álvaro Obregón,CDMX,19,0,756.684211,,
1,1RA VICTORIA,1150,2061.619503,19.386686,-99.201985,Álvaro Obregón,CDMX,29,0,1033.310345,,
2,1RA VICTORIA SECCION BOSQUES,1150,1788.360000,19.387826,-99.197446,Álvaro Obregón,CDMX,74,0,944.648649,,
3,2DA JALALPA TEPITO (AMPL),1260,1063.626266,19.375080,-99.233736,Álvaro Obregón,CDMX,3,0,506.333333,,
4,2DA EL PIRUL (AMPL),1210,1320.710000,19.379745,-99.242231,Álvaro Obregón,CDMX,11,0,878.545455,,


In [29]:
catastro_df_ml['no_airbnb'] = pd.to_numeric(catastro_df_ml['no_airbnb'], downcast="integer")
catastro_df_ml['no_starbucks'] = pd.to_numeric(catastro_df_ml['no_starbucks'], downcast="integer")
catastro_df_ml['airbnb_mean_price'] = pd.to_numeric(catastro_df_ml['airbnb_mean_price'], downcast="integer")
catastro_df_ml['flag_airbnb'] = pd.to_numeric(catastro_df_ml['flag_airbnb'], downcast="integer")
catastro_df_ml['flag_starbucks'] = pd.to_numeric(catastro_df_ml['flag_starbucks'], downcast="integer")

catastro_df_ml['airbnb_mean_price'] = catastro_df_ml['airbnb_mean_price'].fillna(0)

# Generate Multiple Regression Model

In [52]:
# Read processed csv file
catastro_df_ml = pd.read_csv('catastro_ml_model.csv')
catastro_df_ml = catastro_df_ml.fillna(0)
catastro_df_ml

,nombre_col,num_code,prom_valor_unitario_suelo,latitud,longitud,alcaldia,estado,no_airbnb,no_starbucks,airbnb_mean_price,flag_airbnb,flag_starbucks
0,19 DE MAYO,1210,1404.400000,19.361649,-99.253530,Álvaro Obregón,CDMX,19,6,756.684211,1,1
1,1RA VICTORIA,1150,2061.619503,19.386686,-99.201985,Álvaro Obregón,CDMX,29,0,1033.310345,1,0
2,1RA VICTORIA SECCION BOSQUES,1150,1788.360000,19.387826,-99.197446,Álvaro Obregón,CDMX,74,4,944.648649,1,1
3,2DA JALALPA TEPITO (AMPL),1260,1063.626266,19.375080,-99.233736,Álvaro Obregón,CDMX,3,0,506.333333,1,0
4,2DA EL PIRUL (AMPL),1210,1320.710000,19.379745,-99.242231,Álvaro Obregón,CDMX,11,0,878.545455,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1784,GUADALUPE,16900,420.703636,19.218264,-99.120799,Xochimilco,CDMX,0,0,0.000000,0,0
1785,NATIVITAS LA JOYA (AMPL),16900,711.580672,19.237250,-99.093782,Xochimilco,CDMX,2,0,391.500000,1,0
1786,SAN FRANCISCO TLALNEPANTLA (PBLO),16910,338.647387,19.197937,-99.122385,Xochimilco,CDMX,0,0,0.000000,0,0
1787,SANTA CECILIA TEPETLAPA (PBLO),16880,534.238763,19.217187,-99.099325,Xochimilco,CDMX,2,0,5009.000000,1,0


In [53]:
# Enconde categorical columns
le = LabelEncoder()
catastro_encode = catastro_df_ml.copy()
catastro_encode['alcaldia'] = le.fit_transform(catastro_encode['alcaldia'])

In [54]:
# Declare independent variables

X = catastro_encode[['no_starbucks','no_airbnb','alcaldia','airbnb_mean_price']]

# Declare depedant variable to predict
y = catastro_encode['prom_valor_unitario_suelo']

# Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# Create model
model = LinearRegression()

# Fit Model
model.fit(X_train, y_train)

# Generate predictions
y_pred = model.predict(X_test)


# Generate r squared score
#r_squared = model.score(X_train, y_pred)

# Asses accuracy of the model
print('Mean Square Error: ', mean_squared_error(y_test,y_pred))
print('Mean Absolute Error:', mean_absolute_error(y_test,y_pred))
#print('R squared:', r_squared)


Mean Square Error:  2594764.41209991
Mean Absolute Error: 1068.1585560134279


In [55]:
y_pred_total = model.predict(X)

In [56]:
catastro_df_ml['valor_unitario_pred'] = y_pred_total.tolist()

In [57]:
catastro_df_ml

,nombre_col,num_code,prom_valor_unitario_suelo,latitud,longitud,alcaldia,estado,no_airbnb,no_starbucks,airbnb_mean_price,flag_airbnb,flag_starbucks,valor_unitario_pred
0,19 DE MAYO,1210,1404.400000,19.361649,-99.253530,Álvaro Obregón,CDMX,19,6,756.684211,1,1,4746.374323
1,1RA VICTORIA,1150,2061.619503,19.386686,-99.201985,Álvaro Obregón,CDMX,29,0,1033.310345,1,0,1602.655192
2,1RA VICTORIA SECCION BOSQUES,1150,1788.360000,19.387826,-99.197446,Álvaro Obregón,CDMX,74,4,944.648649,1,1,3761.361499
3,2DA JALALPA TEPITO (AMPL),1260,1063.626266,19.375080,-99.233736,Álvaro Obregón,CDMX,3,0,506.333333,1,0,1572.603150
4,2DA EL PIRUL (AMPL),1210,1320.710000,19.379745,-99.242231,Álvaro Obregón,CDMX,11,0,878.545455,1,0,1581.148991
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1784,GUADALUPE,16900,420.703636,19.218264,-99.120799,Xochimilco,CDMX,0,0,0.000000,0,0,1625.301504
1785,NATIVITAS LA JOYA (AMPL),16900,711.580672,19.237250,-99.093782,Xochimilco,CDMX,2,0,391.500000,1,0,1626.442113
1786,SAN FRANCISCO TLALNEPANTLA (PBLO),16910,338.647387,19.197937,-99.122385,Xochimilco,CDMX,0,0,0.000000,0,0,1625.301504
1787,SANTA CECILIA TEPETLAPA (PBLO),16880,534.238763,19.217187,-99.099325,Xochimilco,CDMX,2,0,5009.000000,1,0,1611.034536


In [61]:
catastro_df_ml.to_csv('catastro_ml_model_radio700_results.csv', index = False)

# Classification model testing

## Decision Tree Models

### Decision Tree Classifier for Airbnbs

In [62]:
# Encode categorical columns
le = LabelEncoder()
catastro_encode = catastro_df_ml.copy()
catastro_encode['alcaldia'] = le.fit_transform(catastro_encode['alcaldia'])
catastro_encode['nombre_col'] = le.fit_transform(catastro_encode['nombre_col'])
catastro_encode = catastro_encode.fillna(0)

# Define feature set and target set
X = catastro_encode.copy()
X = X.drop(['estado','flag_airbnb','latitud','longitud','valor_unitario_pred'], axis = 1)
y = catastro_encode['flag_airbnb'].values

# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# Scale the training and testing data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Fit the decision tree model
model = tree.DecisionTreeClassifier()
model = model.fit(X_train_scaled, y_train)

# Generate Predictions
predictions = model.predict(X_test_scaled)

# Evaluate the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index = ['Actual 0', 'Actual 1'], columns =['Predicted 0','Predicted 1'])

# Calculate accuracy score
acc_score = accuracy_score(y_test, predictions)

# Display Results
print('Decision Tree Results')
print('Confusion Matrix')
display(cm_df)
print(f'Accuracy Score:{acc_score}')
print('Classification Report')
print(classification_report(y_test, predictions))

Decision Tree Results
Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,22,0
Actual 1,0,336


Accuracy Score:1.0
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        22
           1       1.00      1.00      1.00       336

    accuracy                           1.00       358
   macro avg       1.00      1.00      1.00       358
weighted avg       1.00      1.00      1.00       358



### Decision Tree Classifier for Starbucks

In [63]:
# Define feature set and target set
X = catastro_encode.copy()
X = X.drop(['estado','flag_starbucks','latitud','longitud'], axis = 1)
y = catastro_encode['flag_starbucks'].values

# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

# Scale the training and testing data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Fit the decision tree model
model = tree.DecisionTreeClassifier()
model = model.fit(X_train_scaled, y_train)

# Generate Predictions
predictions = model.predict(X_test_scaled)

# Evaluate the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index = ['Actual 0', 'Actual 1'], columns =['Predicted 0','Predicted 1'])

# Calculate accuracy score
acc_score = accuracy_score(y_test, predictions)

# Display Results
print('Confusion Matrix')
display(cm_df)
print(f'Accuracy Score:{acc_score}')
print('Classification Report')
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,235,0
Actual 1,0,123


Accuracy Score:1.0
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       235
           1       1.00      1.00      1.00       123

    accuracy                           1.00       358
   macro avg       1.00      1.00      1.00       358
weighted avg       1.00      1.00      1.00       358



## Random Forest Models

### Random Forest Classifier for Airbnbs

In [52]:
# Encode categorical columns
le = LabelEncoder()
catastro_encode = catastro_df.copy()
catastro_encode['alcaldia'] = le.fit_transform(catastro_encode['alcaldia'])

# Define feature set and target set
X = catastro_encode.copy()
X = X.drop(['nombre_col','estado','flag_airbnb','latitud','longitud'], axis = 1)
y = catastro_encode['flag_airbnb'].values

# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

# Scale the training and testing data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create a Random Forest Classifier Instance
rf_model_bnb = RandomForestClassifier(n_estimators = 128, random_state = 72)

# Fit the model
rf_model_bnb = rf_model_bnb.fit(X_train_scaled, y_train)

# Make predictions using the testing data
predictions = rf_model_bnb.predict(X_test_scaled)

# Evaluate the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index = ['Actual 0', 'Actual 1'], columns =['Predicted 0','Predicted 1'])

# Display Results
print('Random Forest Classifier Results')
print('Confusion Matrix')
display(cm_df)
print(f'Accuracy Score:{acc_score}')
print('Classification Report')
print(classification_report(y_test, predictions))

Random Forest Classifier Results
Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,25,0
Actual 1,0,333


Accuracy Score:1.0
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        25
           1       1.00      1.00      1.00       333

    accuracy                           1.00       358
   macro avg       1.00      1.00      1.00       358
weighted avg       1.00      1.00      1.00       358



### Random Forest Classifier for Starbucks

In [53]:
# Define feature set and target set
X = catastro_encode.copy()
X = X.drop(['nombre_col','estado','flag_starbucks','latitud','longitud'], axis = 1)
y = catastro_encode['flag_starbucks'].values

# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

# Scale the training and testing data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create a Random Forest Classifier Instance
rf_model_starbucks = RandomForestClassifier(n_estimators = 128, random_state = 72)

# Fit the model
rf_model_starbucks = rf_model_starbucks.fit(X_train_scaled, y_train)

# Make predictions using the testing data
predictions = rf_model_starbucks.predict(X_test_scaled)

# Evaluate the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index = ['Actual 0', 'Actual 1'], columns =['Predicted 0','Predicted 1'])

# Display Results
print('Random Forest Classifier Results')
print('Confusion Matrix')
display(cm_df)
print(f'Accuracy Score:{acc_score}')
print('Classification Report')
print(classification_report(y_test, predictions))

Random Forest Classifier Results
Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,235,0
Actual 1,0,123


Accuracy Score:1.0
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       235
           1       1.00      1.00      1.00       123

    accuracy                           1.00       358
   macro avg       1.00      1.00      1.00       358
weighted avg       1.00      1.00      1.00       358



## Gradient Boosting Model

### Gradient Boosting Classifier for Airbnbs

In [ ]:
# Encode categorical columns
le = LabelEncoder()
catastro_encode = catastro_df.copy()
catastro_encode['alcaldia'] = le.fit_transform(catastro_encode['alcaldia'])
catastro_encode = catastro_encode.drop(columns = ['nombre_col','Unnamed: 0','estado'])

# Define feature set and target set
X = catastro_encode.copy()
X = X.drop('flag_airbnb', axis = 1)
y = catastro_encode['flag_airbnb'].values

# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

# Scale the training and testing data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Try Different Learning Rates
learning_rates = [0.05,0.07,0.1,0.15,0.20,0.25,0.30,0.35,0.40,
                  0.45,0.5,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95,1]

for learning_rates in learning_rates:
    
    # Iterate Gradient Booster Model
    classifier = GradientBoostingClassifier(n_estimators = 20, learning_rate = learning_rates,
                                            max_features = 9, max_depth = 3, random_state = 0)
    
    # Fit the Model
    classifier.fit(X_train_scaled, y_train)
    
    # Print Report
    print('Learning Rate: ', learning_rate)
    print('Accuracy Score (training): {0:.3f}'.format(classifier.score(X_train_scaled, y_train)))
    print('Accuracy Score (Validation): {0:.3f}'.format(classifier.score(X_test_scaled, y_test)))

In [ ]:
# Select the Learning Rate with better results
classifier = GradientBoostingClassifier(n_estimators = 20, learning_rate = #TBD,
                                        ,max_features = 9, max_depth = 3, random_state = 0)

classifier.fit(X_train_scaled, y_train)
predictions = classifier.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)
print(f'Accuracy Score: {acc_score}')

cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix
cm_df = pd.DataFrame(cm, index = ['Actual 0', 'Actual 1'], columns =['Predicted 0','Predicted 1'])

# Display Results
print('Gradient Boosting Classifier Results')
print('Confusion Matrix')
display(cm_df)
print(f'Accuracy Score:{acc_score}')
print('Classification Report')
print(classification_report(y_test, predictions))

### Gradient Boosting Classifier for Starbucks

In [ ]:
# Encode categorical columns
le = LabelEncoder()
catastro_encode = catastro_df.copy()
catastro_encode['alcaldia'] = le.fit_transform(catastro_encode['alcaldia'])
catastro_encode = catastro_encode.drop(columns = ['nombre_col','Unnamed: 0','estado'])

# Define feature set and target set
X = catastro_encode.copy()
X = X.drop('flag_starbucks', axis = 1)
y = catastro_encode['flag_starbucks'].values

# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

# Scale the training and testing data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Try Different Learning Rates
learning_rates = [0.05,0.07,0.1,0.15,0.20,0.25,0.30,0.35,0.40,
                  0.45,0.5,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.95,1]

for learning_rates in learning_rates:
    
    # Iterate Gradient Booster Model
    classifier = GradientBoostingClassifier(n_estimators = 20, learning_rate = learning_rates,
                                            max_features = 9, max_depth = 3, random_state = 0)
    
    # Fit the Model
    classifier.fit(X_train_scaled, y_train)
    
    # Print Report
    print('Learning Rate: ', learning_rate)
    print('Accuracy Score (training): {0:.3f}'.format(classifier.score(X_train_scaled, y_train)))
    print('Accuracy Score (Validation): {0:.3f}'.format(classifier.score(X_test_scaled, y_test)))

In [ ]:
# Select the Learning Rate with better results
classifier = GradientBoostingClassifier(n_estimators = 20, learning_rate = #TBD,
                                        ,max_features = 9, max_depth = 3, random_state = 0)

classifier.fit(X_train_scaled, y_train)
predictions = classifier.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)
print(f'Accuracy Score: {acc_score}')

cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix
cm_df = pd.DataFrame(cm, index = ['Actual 0', 'Actual 1'], columns =['Predicted 0','Predicted 1'])

# Display Results
print('Gradient Boosting Classifier Results')
print('Confusion Matrix')
display(cm_df)
print(f'Accuracy Score:{acc_score}')
print('Classification Report')
print(classification_report(y_test, predictions))